In [303]:
from z3 import *

# SMT Representation

## The Buffer
### Variables
* $v_{t,i}$ denotes whether the $i^\mathrm{th}$ position in the buffer has a packet at time $t$. It has a maximum length of $buf$
* $s_{t,i}$ denotes the sequence number of that packet (0 means nil)

### Properties
* The buffer is filled from front (i = 0) to back. Hence, $\forall j < i, v_{t, i} = 1 \Rightarrow v_{t, j} = 1$ and $\forall j > i, v_{t, i} = 0 \Rightarrow v_{t, j} = 0$

## The non-deterministic server
* $serve_t = 1$ if the server sends a packet at time t (not modelling losses yet)
* Constraint: $\sum_{t = n T}^{(n + 1)T - 1} serve_t = k$

## The sender
* It sends $send_t = 1$ if it needs to send a packet at time $t$. 
* $seq_i$ denotes the sequence number of the packet to send if $send_t = 1$ else it is the sequence number of the last sent packet
* $lastAck_{t-1}$ is the sequence number of last acked packet (note: there is no reodering)
* $inFlight_t$ is the number of inflight packets. $inFlight_t = seq_{t-1} - lastack_{t}$ since there is no reordering (we use $t-1$ for $seq$, since $inFlight_t$ will determine $seq_t$ and we don't want a circular dependency)

In [578]:
# Parameters

# Time interval (in ticks)
T = 10
# Packets per T
k = 5 #10
# Size of the buffer (in packets)
bufsize = 10
# Propagation delay (in ticks)
rtt_prop = T
# Number of time intervals (T s) we'll simulate
num_T = 4
# Let's start with a constant cwnd for now
cwnd = 8 #10

In [579]:
# Declare all our variables

# Buffer variables
v, s = [], []
for t in range(num_T * T):
    v += [[Bool('v_{%d,%d}' % (t, i)) for i in range(bufsize)]]
    s += [[Int('v_{%d,%d}' % (t, i)) for i in range(bufsize)]]

# Sender variables
send = [Bool('send_%d' % t) for t in range(num_T * T)]
seq = [Int('seq_%d' % t) for t in range(num_T * T)]

# Server variables
serve = [Bool('serve_%d' % t) for t in range(num_T * T)]
lastAck = [Int('lastAck_%d' % t) for t in range(num_T * T)]
inFlight = [Int('inFlight_%d' % t) for t in range(num_T * T)]

In [584]:
solver = Solver()

# The server constraints
for n in range(num_T):
    solver.add(Sum([If(x, 1, 0) for x in serve[n * T:(n + 1) * T]]) == k)

# Buffer constraints to set v and s
for i in range(0, bufsize):
    # Initial conditions
    solver.add(v[0][i] == False)
    solver.add(s[0][i] == 0)
for t in range(1, num_T * T):
    for i in range(0, bufsize):
        if i > 0:
            # Add packet at the i^th position at time t if v_{t-1,i} = 0 and v_{t-1,i-1} = 1
            add_pkt = And(send[t], v[t - 1][i] == False,
                          v[t - 1][i - 1] == True)
        else:
            add_pkt = And(send[t], v[t - 1][0] == False)
        # Note: If buffer were full, we won't add and the packet will be dropped automatically

        # Should we move packets because we served a packet?
        move = And(serve[t], v[t-1][0] == True)

        # Add packet
        solver.add(
            Implies(And(add_pkt, Not(move)),
                    And(v[t][i] == True, s[t][i] == seq[t])))
        # Note, if move is True, we set i-1, otherwise we set i
        solver.add(
            Implies(And(add_pkt, move),
                    And(v[t][i - 1] == True, s[t][i - 1] == seq[t])))

        # Move packets if we are serving
        if i > 0: #i < bufsize - 1:
            solver.add(
                Implies(
                    And(Not(add_pkt), move),
                    And(v[t][i - 1] == v[t - 1][i],
                        s[t][i - 1] == s[t - 1][i])))
            #solver.add(
            #    Implies(
            #        And(Not(add_pkt), move),
            #        And(v[t][i] == v[t - 1][i + 1],
            #            s[t][i] == s[t - 1][i + 1])))
        # Not adding a packet, not moving
        solver.add(
            Implies(And(Not(add_pkt), Not(move)),
                    And(v[t][i] == v[t - 1][i], s[t][i] == s[t - 1][i])))

        # Boundary condition: when moving packets, the last slot is unfilled unless a packet was just added
        solver.add(
            Implies(And(Not(add_pkt), move), And(v[t][bufsize - 1] == False)))

        # If a packet isn't there (i.e. v_{t, i} = 0), set the s_{t, i} = 0. This isn't necessary
        # for correctness, but may make the search space simpler (who knows?)
        solver.add(Implies(Not(v[t][i]), s[t][i] == 0))

# Calculate the lastAck, seq and inFlight
solver.add(lastAck[0] == 0)
solver.add(inFlight[0] == 0)
for t in range(1, num_T * T):
    if t >= rtt_prop:
        # Did we just receive an ack?
        acked = And(serve[t - rtt_prop + 1], v[t - rtt_prop][0])

        # Update lastAck accordingly
        solver.add(Implies(acked, lastAck[t] == s[t - rtt_prop][0]))
        solver.add(Implies(Not(acked), lastAck[t] == lastAck[t - 1]))
    else:
        solver.add(lastAck[t] == lastAck[t - 1])

    # Set inFlight
    solver.add(inFlight[t] == seq[t - 1] - lastAck[t])

# Implement the sender according to a cwnd condition
solver.add(send[0] == False)
solver.add(seq[0] == 0)
for t in range(1, num_T * T):
    # Should we transmit?
    transmit = cwnd > inFlight[t]

    # Set send and seq
    solver.add(
        Implies(transmit, And(send[t] == True, seq[t] == seq[t - 1] + 1)))
    solver.add(
        Implies(Not(transmit), And(send[t] == False, seq[t] == seq[t - 1])))

# Performance constraint
solver.add(lastAck[-1] < 13)

In [585]:
solver.check()

sat

In [586]:
def display(solver):
    model = solver.model()
    for t in range(num_T * T):
        line = "{:<4}".format(t)
        line += ('    ', 'Send')[int(bool(model[send[t]]))]
        
        line += " " * 5
        
        for i in range(bufsize-1, -1, -1):
            if model[s[t][i]].as_long() == 0:
                line += ' ' * 5
            else:
                line += "{:5}".format(model[s[t][i]].as_long())
            assert(not ((not bool(model[v[t][i]])) ^ model[s[t][i]].as_long() == 0))

        line += "  :  "
        
        if model[serve[t]]:
            line += "Served"
        else:
            line += "      "
        
        line += "  inFlight=%s lastAck=%s" % (model[inFlight[t]], model[lastAck[t]])
        
        print(line)

display(solver)

0                                                                :  Served  inFlight=0 lastAck=0
1   Send                                                      1  :  Served  inFlight=0 lastAck=0
2   Send                                                 2    1  :          inFlight=1 lastAck=0
3   Send                                                 3    2  :  Served  inFlight=2 lastAck=0
4   Send                                                 4    3  :  Served  inFlight=3 lastAck=0
5   Send                                            5    4    3  :          inFlight=4 lastAck=0
6   Send                                            6    5    4  :  Served  inFlight=5 lastAck=0
7   Send                                       7    6    5    4  :          inFlight=6 lastAck=0
8   Send                                  8    7    6    5    4  :          inFlight=7 lastAck=0
9                                         8    7    6    5    4  :          inFlight=8 lastAck=0
10                            

In [424]:
solver.statistics()

(:arith-bound-propagations-cheap 141
 :arith-bound-propagations-lp    176
 :arith-conflicts                3
 :arith-lower                    365
 :arith-make-feasible            58
 :arith-max-columns              132
 :arith-max-rows                 73
 :arith-propagations             141
 :arith-rows                     698
 :arith-upper                    325
 :binary-propagations            1056
 :conflicts                      15
 :decisions                      36
 :eliminated-vars                59
 :final-checks                   1
 :max-memory                     247.60
 :memory                         45.23
 :minimized-lits                 3
 :mk-bool-var                    576
 :mk-clause                      655
 :num-allocs                     153456121
 :num-checks                     1
 :propagations                   1429
 :rlimit-count                   926221273)